In [1]:
import sys
sys.path.append('../src/')

import warnings
#warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

import os
from pathlib import Path
import json
import pickle

from session import Session
from dppa import DPPA
from helpers import *

In [2]:
FAMILY_ID = 4
SEANCE_ID = 1
SESSION_ID = 0
session_pathname = Path(f"../data/output/processed/session_{SESSION_ID}_family_{FAMILY_ID}_seance_{SEANCE_ID}.pkl")

with open(session_pathname, "rb") as f:
    session = pickle.load(f)
print(f"Session loaded from {session_pathname}")

for pr in session.physio_recordings:
    print(f"Signal epoched for subject {pr.subject_id}: {pr.data_epoched}")

Session loaded from ../data/output/processed/session_0_family_4_seance_1.pkl
Signal epoched for subject 0: True
Signal epoched for subject 1: True
Signal epoched for subject 2: True


In [3]:
dppa = DPPA()
dppa.set_session(session)

DPPA initialized. Ready to set session and compute features.
Session 0 set with 3 physio recordings. Features initialized.


In [4]:
for pr in session.physio_recordings:
    print(f"Loaded RR intervals epochs for subject {pr.subject_id}")
    print(f"\tLoaded {len(dppa.features["subjects"][pr.subject_id]["RR_Intervals"]["rs"])} epochs for resting state")
    print(f"\tLoaded {len(dppa.features["subjects"][pr.subject_id]["RR_Intervals"]["session"])} epochs for session")

Loaded RR intervals epochs for subject 0
	Loaded 1 epochs for resting state
	Loaded 165 epochs for session
Loaded RR intervals epochs for subject 1
	Loaded 1 epochs for resting state
	Loaded 165 epochs for session
Loaded RR intervals epochs for subject 2
	Loaded 1 epochs for resting state
	Loaded 165 epochs for session


In [5]:
dppa.run()

Running DPPA computation...
Computing individual features for each subject...
	Computed features for subject 0: Centroid, SD1, SD2, and SD1/SD2 Ratio.
	Computed features for subject 1: Centroid, SD1, SD2, and SD1/SD2 Ratio.
	Computed features for subject 2: Centroid, SD1, SD2, and SD1/SD2 Ratio.
Individual features computed.
Computing Inter-Centroid Distances (ICD) for all dyads...
	Computed ICD for dyad (0, 1) in both rs and session steps.
	Computed ICD for dyad (0, 2) in both rs and session steps.
	Computed ICD for dyad (1, 2) in both rs and session steps.
ICD computed.
DPPA computation completed.


In [6]:
dppa.compute_clusters(threshold = 100)

Computing clusters of subjects based on Centroid features...
Clusters computed:
	2-clusters: {'0_1': 0.11515151515151516, '0_2': 0.26666666666666666, '1_2': 0.1393939393939394}
	3-soft-clusters: 0.11515151515151516
	3-strong-clusters: 0.21818181818181817


In [7]:
dppa_pathname = Path(f"../data/output/dppa/session_{SESSION_ID}_family_{FAMILY_ID}_seance_{SEANCE_ID}.pkl")
if not dppa_pathname.parent.exists():
    os.makedirs(dppa_pathname.parent)

with open(dppa_pathname, "wb") as f:
    pickle.dump(dppa, f)
print(f"DPPA saved to {dppa_pathname}")

DPPA saved to ../data/output/dppa/session_0_family_4_seance_1.pkl
